In [117]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [118]:
data = pd.read_csv("train.csv")
data = data.drop(columns=["case_id", "Hospital_code", "City_Code_Hospital","patientid"])

#data = data.select_dtypes(exclude=[np.number])

data = data.sample(frac=.01)
len(data)


3184

In [119]:
data["cat_available_rooms"] = pd.cut(data["Available Extra Rooms in Hospital"].values, bins = 6,labels = [0,1,2,3,4,5])
data["cat_bed_grade"] = pd.cut(data["Bed Grade"].values, bins = 6,labels = [0,1,2,3,4,5])
data["cat_city_code_patient"] = pd.cut(data["City_Code_Patient"].values, bins = 6,labels = [0,1,2,3,4,5])
data["cat_visitors_with_patient"] = pd.cut(data["Visitors with Patient"].values, bins = 6,labels = [0,1,2,3,4,5])
data["cat_admission_deposit"] = pd.cut(data["Admission_Deposit"].values, bins = 6,labels = [0,1,2,3,4,5])

data = data.drop(columns=["Available Extra Rooms in Hospital","Bed Grade","City_Code_Patient","Visitors with Patient","Admission_Deposit"])
data = data[["cat_available_rooms","cat_bed_grade","cat_city_code_patient","cat_visitors_with_patient","cat_admission_deposit","Hospital_type_code","Hospital_region_code","Department","Ward_Type"
            ,"Ward_Facility_Code","Type of Admission","Severity of Illness","Age","Stay"
            ]]

data.head(1)

,cat_available_rooms,cat_bed_grade,cat_city_code_patient,cat_visitors_with_patient,cat_admission_deposit,Hospital_type_code,Hospital_region_code,Department,Ward_Type,Ward_Facility_Code,Type of Admission,Severity of Illness,Age,Stay
112371,1,3,0,0,2,a,Y,gynecology,R,C,Trauma,Extreme,81-90,21-30


In [120]:
#Calculate P(Y=y) for all possible y
def calculate_prior(df, Y):
    classes = sorted(list(df[Y].unique()))
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

In [121]:
def calculate_likelihood_categorical(df, feat_name, feat_val, Y, label):
    feat = list(df.columns)
    df = df[df[Y]==label]
    p_x_given_y = len(df[df[feat_name]==feat_val]) / len(df)
    return p_x_given_y

In [122]:
def naive_bayes_categorical(df, X, Y):
    # get feature names
    features = list(df.columns)[:-1]

    # calculate prior
    prior = calculate_prior(df, Y)

    Y_pred = []
    # loop over every data sample
    for x in X:
        # calculate likelihood
        labels = sorted(list(df[Y].unique()))
        likelihood = [1]*len(labels)
        for j in range(len(labels)):
            for i in range(len(features)):
                likelihood[j] *= calculate_likelihood_categorical(df, features[i], x[i], Y, labels[j])

        # calculate posterior probability (numerator only)
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]
        
        Y_pred.append(labels[np.argmax(post_prob)])
    
    return np.array(Y_pred) 

In [123]:
def compute_accuracy(y_true, y_pred):
    correct_predictions = 0
    # iterate over each label and check
    for true, predicted in zip(y_true, y_pred):
        if true == predicted:
            correct_predictions += 1
    # compute the accuracy
    accuracy = correct_predictions/len(y_true)
    return accuracy

In [124]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=.1)

X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values
Y_pred = naive_bayes_categorical(train, X=X_test, Y="Stay")

print(compute_accuracy(Y_test,Y_pred))

#Current accuracy => 0.322884012539185

0.322884012539185
